使用indritop50进行训练，标签使用人工标注的标签， 如果没有标注则认为是负样本

In [1]:
#加载indri数据集
indri_dic={}
with open('2009-2012.diversity.indri','r') as f:
    for line in f:
        parts=line.split(' ')
        if(len(parts)==6):
            qid,_,docid,_,_,_=parts
            if qid not in indri_dic:
                indri_dic[qid]=[]
            if len(indri_dic[qid])>=100:#50:
                continue
            indri_dic[qid].append(docid)
 

In [2]:
import pickle
with open('full_content.pkl','rb') as f:
    docs=pickle.load(f)
print(type(docs))

<class 'dict'>


In [3]:
import json
with open('subtopics_suggestions.json','r') as f:
    query_and_subtopic=json.load(f)

In [4]:
#加载人工标注的数据
data_dict = {}
seq_dict={}
with open('2009-2012.diversity.ndeval.qrels', 'r') as file:
    for line in file:
        parts = line.strip().split()
        # 确保每行有足够的列
        if len(parts) >= 4:
            query_id, _, doc_id, label = parts
            if query_id not in data_dict:
                data_dict[query_id]={}
            if query_id not in seq_dict:
                seq_dict[query_id]=[]
            seq_dict[query_id].append([doc_id,label])
            if doc_id not in data_dict[query_id]:
                data_dict[query_id][doc_id]=0
            if label!='0':
                data_dict[query_id][doc_id]+=1

In [5]:
data_dict={}
for qid in seq_dict:
    if qid not in data_dict:
        data_dict[qid]={}
    for item in seq_dict[qid]:
        docid,label=item
        label=int(label)
        if docid not in data_dict[qid]:
            data_dict[qid][docid]=0
        data_dict[qid][docid]+=label

In [6]:
for qid,doc_list in indri_dic.items():
    #标注里面没有这个queryid
    if qid not in data_dict:
        continue
    for doc_id in doc_list:
        if doc_id not in data_dict[qid]:
            data_dict[qid][doc_id]=0
    data_dict[qid]=sorted(data_dict[qid].items(),key=lambda x:x[1],reverse=True)

In [7]:
len(data_dict['1'])

520

In [8]:
samples_list=[]
import random
random.seed(2024)
max_length=512
subtopic_content_dict={}
for qid in data_dict:
    doc_list=data_dict[qid]
    pos_list=[item for item in doc_list if item[1]>0]
    neg_list=[item for item in doc_list if item[1]==0]
    query_content=query_and_subtopic[qid]['query']
    subtopic=query_and_subtopic[qid]['subtopic']
    subtopic_content_dict[query_content]={"subtopics":subtopic.split(",")}
    for i in range(len(pos_list)):
        pos_id,pos_label=pos_list[i]
        if pos_id not in docs:
            continue
        pos=[''.join(docs[pos_id][:max_length])]
        negs=[]
        for j in range(i+1,len(pos_list)):

            neg_id,neg_label=pos_list[j]
            if pos_label>neg_label:
                if neg_id not in docs:
                    continue
                neg="".join(docs[neg_id][:max_length])
                negs.append(neg)
        if len(negs)>0:
            samples_list.append([query_content,pos,negs])
    for pos_item in pos_list:
        random.shuffle(neg_list)
        #neg_list_sampled=neg_list[:10]
        neg_list_sampled=neg_list
        pos_id=pos_item[0]
        if pos_id not in docs:
            continue
        pos=[''.join(docs[pos_id][:max_length])]
        negs=[]
        for neg_item in neg_list_sampled:
            neg_id=neg_item[0]
            if neg_id not in docs:
                continue
            neg="".join(docs[neg_id][:max_length])
            negs.append(neg)
        if len(negs)>0:
            samples_list.append([query_content,pos,negs])

In [9]:
prompt_former="Consider a query A and a passage B. The subtopics of this query are as follows:"
prompt_latter="These subtopics are specific and subdivided topics related to the query. Please determine whether passage B contains possible intents related to query A, and give the answer by predicting 'yes' or 'no'"
# prompt="Consider a query A and a passage B. The subtopics of this query are as follows: 1.xxx, 2.xxx. These subtopics are specific and subdivided topics related to the query. Please determine whether passage B contains possible intents related to query A, and give the answer by predicting 'yes' or 'no'"
import json
with open("../finetune_data_bge_jsonl/subtopic_top100.jsonl",'w') as f:
    for item in samples_list:
        query,pos,neg=item
        assert isinstance(pos,list)
        assert isinstance(neg,list)
        
        subtopics=subtopic_content_dict[query]['subtopics']
        prompt=""+prompt_former

        for index,subtopic in enumerate(subtopics):
            number=index+1
            subtopic=" ".join(subtopic.split("_"))
            if number==len(subtopics):
                prompt=prompt+f"{number}.'"+subtopic+"'. "
            else:
                prompt=prompt+f"{number}.'"+subtopic+"', "

        prompt=prompt+prompt_latter
        

        dict_write={"query":query,"pos":pos,"neg":neg,"prompt":prompt}
        write_line=json.dumps(dict_write)
        f.write(write_line+'\n')


In [10]:
len(samples_list)

6177

In [46]:
dict_pos_neg_list={}
for item in samples_list:
    query,pos,neg=item
    if query not in dict_pos_neg_list:
        dict_pos_neg_list[query]={}
    if pos not in dict_pos_neg_list[query]:
        dict_pos_neg_list[query][pos]=set()
    dict_pos_neg_list[query][pos].add(neg)


In [50]:
length_list=[len(dict_pos_neg_list[item]) for item in dict_pos_neg_list]

In [51]:
length_list

[1,
 1,
 1,
 2,
 6,
 2,
 11,
 2,
 3,
 59,
 28,
 3,
 32,
 6,
 7,
 12,
 30,
 3,
 19,
 21,
 1,
 7,
 15,
 29,
 1,
 2,
 9,
 18,
 25,
 28,
 2,
 20,
 5,
 1,
 15,
 13,
 12,
 15,
 22,
 26,
 17,
 12,
 47,
 6,
 4,
 5,
 37,
 15,
 4,
 23,
 3,
 15,
 61,
 23,
 4,
 53,
 37,
 54,
 10,
 40,
 2,
 5,
 7,
 18,
 19,
 7,
 2,
 29,
 24,
 50,
 38,
 17,
 7,
 2,
 43,
 20,
 9,
 49,
 20,
 35,
 20,
 14,
 35,
 30,
 3,
 21,
 11,
 19,
 11,
 20,
 19,
 33,
 19,
 36,
 16,
 20,
 8,
 10,
 32,
 30,
 32,
 57,
 6,
 48,
 10,
 23,
 38,
 61,
 32,
 30,
 40,
 26,
 38,
 23,
 50,
 5,
 20,
 24,
 20,
 57,
 5,
 4,
 10,
 3,
 24,
 12,
 44,
 4,
 8,
 13,
 42,
 2,
 17,
 44,
 14,
 44,
 65,
 44,
 11,
 25,
 26,
 33,
 13,
 23,
 43,
 24,
 67,
 22,
 16,
 6,
 1,
 5,
 3,
 16,
 2,
 9,
 48,
 10,
 16,
 33,
 29,
 34,
 26,
 6,
 2,
 16,
 48,
 15,
 1,
 2,
 15,
 1,
 15,
 2,
 34,
 57,
 4,
 3,
 6,
 32,
 11,
 18,
 41,
 8,
 48,
 10,
 21,
 11,
 37]

In [22]:
from datasets import Dataset

# 假设你已经有了data_list
# ...

data = {"query": [], "subtopic": [],"pos": [], "neg": []}
for item in samples_list:

    data["query"].append(item[0])
    data["subtopic"].append(item[1])
    data["pos"].append(item[2])
    data["neg"].append(item[3])
    

# 创建Hugging Face dataset
dataset = Dataset.from_dict(data)

# 保存数据集到本地
dataset.save_to_disk('pairwise_samples_simplified')

print("Dataset saved successfully.")

Saving the dataset (0/3 shards):   0%|          | 0/46029 [00:00<?, ? examples/s]

Dataset saved successfully.


In [23]:
print(dataset)

Dataset({
    features: ['query', 'subtopic', 'pos', 'neg'],
    num_rows: 46029
})
